In [1]:
import pandas as pd
import json

### Shrink the dataset into subsets

In [14]:
data = []
prs = 0
with open('/Users/osamaalhalabi/Documents/GradProject/RecSys/HotelRec Dataset/HotelRec.json') as f:
    for line in f:
        data.append(json.loads(line))
        
        if len(data) == 5000000:
            df = pd.DataFrame(data)
            df.drop(["author", "date", "title"], axis = 1, inplace = True) 
            df = df[df['property_dict'].map(len) > 2]
            df=df.reset_index(drop= True)
            data.clear()
            prs = prs + 1
            df.to_csv("/Users/osamaalhalabi/Documents/GradProject/RecSys/Parsed Dataset/part_{0}.csv".format(prs),
                      header=True, index=False) 
            print('parsing progress: {0}/10'.format(prs))
            
        if prs == 10:
            break

parsing progress: 1/10
parsing progress: 2/10
parsing progress: 3/10
parsing progress: 4/10
parsing progress: 5/10
parsing progress: 6/10
parsing progress: 7/10
parsing progress: 8/10
parsing progress: 9/10
parsing progress: 10/10


#### Preprocessing the Data

In [194]:
def get_hotel_name(url : str):
    start = url.find('-Reviews-')+9
    mid = url.find('-',start)
    end = url.find('.',mid)
    return url[start:mid], url[mid+1:end]

def change_names(names):
    names_ = [] 
    locs = []
    for hotel in names:
        (name,loc) = get_hotel_name(hotel)
        names_.append(name)
        locs.append(loc)
    return names_,locs

In [274]:
def parse(index):
    df = pd.read_csv("/Users/osamaalhalabi/Documents/GradProject/RecSys/Parsed Dataset/part_{0}.csv".format(index))
    names, locs = change_names(df.hotel_url)
    df.hotel_url = names
    df['address'] = locs
    
    df['hotel_url'] = df['hotel_url'].str.replace('_',' ')
    df['address'] = df['address'].str.replace('_',' ')
    df.rename(columns = {'hotel_url' : 'name' }, inplace = True)
    
    df_rev = df.groupby("name").sample(20, random_state=2 , replace = True)
    df_rev.drop(["address", "property_dict"], axis = 1, inplace = True) 
    df_rev = df_rev.drop_duplicates()
    df_rev.to_csv("/Users/osamaalhalabi/Documents/GradProject/RecSys/Probabilistic Dataset/reviews_{0}.csv".format(index),
                      header=True, index=False) 
    print("Reviews extracting done..")

    dicts = []
    for row in df.property_dict:
        dicts.append(eval(row))
        
    df_rate = pd.json_normalize(dicts)
    df_rate = df_rate.join(df.name, lsuffix="_left", rsuffix="_right")
    df_rate = df_rate.join(df.rating, lsuffix="_left", rsuffix="_right")
    df_rate = df_rate.join(df.address, lsuffix="_left", rsuffix="_right")
    
    df_rate = df_rate.groupby(['name','address'])['rating','service',
                                                   'location','value','cleanliness','sleep quality',
                                                   'check in / front desk', 'rooms',
                                                   'business service (e.g., internet access)'].mean()
    df_rate = df_rate.round(1)
    df_rate.to_csv("/Users/osamaalhalabi/Documents/GradProject/RecSys/Probabilistic Dataset/prob_{0}.csv".format(index),
                      header=True, index=True) 
    print("Ratings extracting done!")
    return  'Done!'

In [276]:
for idx in range(1, 11):
    parse(idx)

Reviews extracting done..


<ipython-input-274-a55b97663588>:27: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_rate = df_rate.groupby(['name','address'])['rating','service',


Ratings extracting done!
Reviews extracting done..
Ratings extracting done!
Reviews extracting done..
Ratings extracting done!
Reviews extracting done..
Ratings extracting done!
Reviews extracting done..
Ratings extracting done!
Reviews extracting done..
Ratings extracting done!
Reviews extracting done..
Ratings extracting done!
Reviews extracting done..
Ratings extracting done!


### Merge Subsets into One Dataset

In [282]:
fout=open("/Users/osamaalhalabi/Documents/GradProject/RecSys/prob_fin.csv","a")
for line in open("/Users/osamaalhalabi/Documents/GradProject/RecSys/Probabilistic Dataset/prob_1.csv"):
    fout.write(line)   
for num in range(2,11):
    f = open("/Users/osamaalhalabi/Documents/GradProject/RecSys/Probabilistic Dataset/prob_"+str(num)+".csv")
    f.__next__() 
    for line in f:
         fout.write(line)
    f.close() 
fout.close()

In [283]:
fout=open("/Users/osamaalhalabi/Documents/GradProject/RecSys/reviews_fin.csv","a")
for line in open("/Users/osamaalhalabi/Documents/GradProject/RecSys/Probabilistic Dataset/reviews_1.csv"):
    fout.write(line)  
for num in range(2,11):
    f = open("/Users/osamaalhalabi/Documents/GradProject/RecSys/Probabilistic Dataset/reviews_"+str(num)+".csv")
    f.__next__() 
    for line in f:
         fout.write(line)
    f.close() 
fout.close()

In [285]:
rdf = pd.read_csv("/Users/osamaalhalabi/Documents/GradProject/RecSys/reviews_fin.csv")
rdf.head()

,name,rating,text
0,0031 Boutique Hotel Restaurant,5.0,At 0031 you come to the right place. Everythin...
1,0031 Boutique Hotel Restaurant,5.0,Just came back from my trip to Cumbuco. It was...
2,0031 Boutique Hotel Restaurant,5.0,I was 6 weeks in Cumbuco. 0031 is the best pla...
3,0031 Boutique Hotel Restaurant,5.0,everything was very nice. we had a lovely stay...
4,0031 Boutique Hotel Restaurant,5.0,For the third time in a row we've been to this...


In [286]:
pdf = pd.read_csv("/Users/osamaalhalabi/Documents/GradProject/RecSys/prob_fin.csv")
pdf.head()

/Users/osamaalhalabi/opt/anaconda3/envs/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,address,rating,service,location,value,cleanliness,sleep quality,check in / front desk,rooms,"business service (e.g., internet access)"
0,0031 Boutique Hotel Restaurant,Cumbuco Caucaia State of Ceara,4.9,4.9,4.6,4.8,4.8,4.9,5.0,4.8,4.5
1,10 Cottages,Franz Josef Westland National Park Te Wahipoun...,4.4,4.5,4.7,4.3,4.7,4.5,NaN,4.3,NaN
2,10 Karakoy Istanbul,Istanbul,4.7,4.7,4.8,4.7,4.9,4.6,NaN,4.7,NaN
3,100 Sunset Boutique Hotel,Kuta Kuta District Bali,4.1,4.3,3.4,4.3,4.1,4.2,NaN,4.2,NaN
4,100 Sunset Hotel Managed by Eagle Eyes,Kuta Kuta District Bali,4.2,4.4,3.6,4.4,4.5,4.4,NaN,4.3,NaN


In [296]:
pdf['name'].count(), rdf['text'].count()

(382086, 4605834)